In [1]:
import pandas as pd
import numpy as np
from h3 import h3

In [2]:
df = pd.read_csv("h3_agg.csv")

In [3]:
df = df.rename(columns={'code_18': "corine_landuse", "percipitation":"precipitation", "Varv": "soil_class"})

In [4]:
df.describe()

,elevation,corine_landuse,temperature,precipitation,population,soil_class
count,124767.000000,124767.000000,124767.000000,124767.000000,124767.000000,88776.000000
mean,39.184832,306.372871,-32.096019,487.100083,8.764933,6.606932
std,37.905403,149.353962,20.975767,278.413802,143.997517,6.853250
min,-0.686404,0.000000,-61.000000,0.000000,0.000000,0.000000
25%,0.256986,211.000000,-50.000000,564.000000,0.000000,1.000000
50%,33.173809,311.000000,-42.000000,621.000000,0.149468,4.000000
75%,62.688461,512.000000,-10.000000,661.000000,1.125973,11.000000
max,278.191096,523.000000,0.000000,783.000000,31795.673828,22.000000


In [5]:
df.sample(3)

,cell_id,elevation,corine_landuse,temperature,precipitation,population,soil_class
119050,881f6591e1fffff,15.499795,311,-39.0,695.0,0.03462,11.0
116712,881f65571dfffff,38.748594,311,-35.0,656.0,0.00000,NaN
104286,881f64871bfffff,0.000000,523,0.0,0.0,0.00000,NaN


In [6]:
def h3_all_parents(base_cell, lowest_res=1):
    res = h3.h3_get_resolution(base_cell)
    t_cell = base_cell
    parents = []
    while res > lowest_res:
        parent = h3.h3_to_parent(t_cell)
        parents.append(parent)
        t_cell = parent
        res = h3.h3_get_resolution(parent)
    return parents


In [7]:
df['parent_ids'] = df['cell_id'].apply(lambda x: ":".join(h3_all_parents(x, lowest_res=2)))

In [8]:
df[f'parent_id_8'] = df['cell_id']

In [9]:
df.sample(5)

,cell_id,elevation,corine_landuse,temperature,precipitation,population,soil_class,parent_ids,parent_id_8
65365,88113464e9fffff,73.915213,231,-53.0,578.0,1.690376,8.0,87113464effffff:86113464fffffff:85113467ffffff...,88113464e9fffff
102762,881f64355bfffff,0.000000,523,0.0,0.0,0.000000,NaN,871f64355ffffff:861f64357ffffff:851f6437ffffff...,881f64355bfffff
19377,88089ab989fffff,26.556912,311,-30.0,657.0,0.120124,4.0,87089ab98ffffff:86089ab9fffffff:85089abbffffff...,88089ab989fffff
62713,88113441dbfffff,33.256639,512,-49.0,655.0,0.000000,0.0,87113441dffffff:86113441fffffff:85113443ffffff...,88113441dbfffff
60496,8811342441fffff,136.527781,312,-55.0,639.0,0.088142,4.0,871134244ffffff:861134247ffffff:85113427ffffff...,8811342441fffff


In [10]:
parent_df = []

variable = "elevation"

for idx in range(7,1,-1):
    print(idx)
    
    df[f'parent_id_{idx}'] = df[f'parent_id_{idx+1}'].map(h3.h3_to_parent)
    
    t8 = df.groupby(f'parent_id_{idx}')[variable].describe()
    t8 = pd.DataFrame(t8).drop(
                        columns=["min","25%","50%","75%","max"]
                    ).rename(columns={
                        'mean':variable, "std": variable + "_" + "std", "count": variable + "_" + "count"
                    })
    t8 = t8.reset_index(drop=False)
    t8['cell_id'] = t8[f'parent_id_{idx}']
    t8['resolution'] = idx
    t8['parent_ids'] = t8['cell_id'].apply(lambda x: ":".join(h3_all_parents(x, lowest_res=2)))
    t8 = t8[['cell_id', 'elevation', 'resolution', 'parent_ids', 'elevation_count','elevation_std']]
    t8['elevation_count'] = pd.to_numeric(t8['elevation_count'], downcast='integer')
    parent_df.append(t8)

7
6
5
4
3
2


In [11]:
subs = pd.concat(parent_df)

In [12]:
subs.sample(5)

,cell_id,elevation,resolution,parent_ids,elevation_count,elevation_std
3953,87089b66affffff,2.360919,7,86089b66fffffff:85089b67fffffff:84089b7fffffff...,7,2.493859
10056,8711349a5ffffff,46.283710,7,8611349a7ffffff:8511349bfffffff:8411349fffffff...,7,0.926130
8531,8711340deffffff,74.030782,7,8611340dfffffff:8511340ffffffff:8411341fffffff...,7,1.807254
12079,871135d56ffffff,83.482802,7,861135d57ffffff:851135d7fffffff:841135dfffffff...,7,1.018790
17984,871f65d9bffffff,0.000000,7,861f65d9fffffff:851f65dbfffffff:841f65dfffffff...,7,0.000000


In [13]:
subs.describe()

,elevation,resolution,elevation_count,elevation_std
count,21168.000000,21168.000000,21168.000000,21106.000000
mean,38.887971,6.822610,35.364796,2.524031
std,37.777596,0.465917,594.947494,3.579622
min,-0.000071,2.000000,1.000000,0.000000
25%,0.982977,7.000000,7.000000,0.023043
50%,32.890419,7.000000,7.000000,1.604654
75%,62.198756,7.000000,7.000000,3.211372
max,259.852890,7.000000,56885.000000,97.862109


In [14]:
elev = df[['cell_id','elevation', 'parent_ids']].copy()
elev['resolution']  = 8
elev['elevation_count']  = 1
elev['elevation_std']  = 0
elev = pd.concat([elev, subs])
elev['elevation_count'] = pd.to_numeric(elev['elevation_count'], downcast='integer')

In [15]:
elev.sample(5)

,cell_id,elevation,parent_ids,resolution,elevation_count,elevation_std
84030,881135da21fffff,96.697370,871135da2ffffff:861135da7ffffff:851135dbffffff...,8,1,0.000000
103852,881f64822bfffff,0.000000,871f64822ffffff:861f64827ffffff:851f6483ffffff...,8,1,0.000000
11936,871135c9bffffff,104.167037,861135c9fffffff:851135cbfffffff:841135dfffffff...,7,7,3.195369
6435,88089a0e6bfffff,19.162474,87089a0e6ffffff:86089a0e7ffffff:85089a0fffffff...,8,1,0.000000
19373,88089ab981fffff,23.069306,87089ab98ffffff:86089ab9fffffff:85089abbffffff...,8,1,0.000000


In [16]:
elev.describe()

,elevation,resolution,elevation_count,elevation_std
count,145935.000000,145935.000000,145935.000000,145873.000000
mean,39.141772,7.829218,5.984644,0.365196
std,37.886907,0.450996,226.907363,1.625518
min,-0.686404,2.000000,1.000000,0.000000
25%,0.396949,8.000000,1.000000,0.000000
50%,33.134811,8.000000,1.000000,0.000000
75%,62.632599,8.000000,1.000000,0.000000
max,278.191096,8.000000,56885.000000,97.862109


In [17]:
elev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145935 entries, 0 to 4
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   cell_id          145935 non-null  object 
 1   elevation        145935 non-null  float64
 2   parent_ids       145935 non-null  object 
 3   resolution       145935 non-null  int64  
 4   elevation_count  145935 non-null  int32  
 5   elevation_std    145873 non-null  float64
dtypes: float64(2), int32(1), int64(1), object(2)
memory usage: 7.2+ MB


In [18]:
import sqlite3

conn = sqlite3.connect('test.db')
c = conn.cursor()

In [19]:
rs = c.execute("select * from dggs_catalog;")

In [20]:
for row in rs:
    print(row)

('srtm_30m_estonia_h3', 'H3', '24,58,27,59', 9, '9:8:7:6:5:4:3:2', 'elevation', '30m SRTM sampled at res 9 covering Estonia', None, 1634280469, None)


In [21]:
rs = c.execute("select * from srtm_30m_estonia_h3;")

In [22]:
for row in rs:
    print(row)

In [23]:
elev.to_sql('srtm_30m_estonia_h3', conn, if_exists='append', index = False)

In [24]:
c.close()

In [25]:
conn.close()

In [27]:
h3.h3_get_resolution("820897fffffffff")

2